### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Language_Abilities/GSE84046'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_age = None  # define the functions when applicable

# 1. Gene Expression Data Availability
is_gene_available = True  # Based on the information provided

# 2. Variable Availability and Data Type Conversion
# Checking for 'Language_Abilities', 'age', and 'gender' in sample characteristics

# For 'Language_Abilities', as there is no clear indication in the provided information, we'll consider it as not available
trait_row = None

# For 'age', assuming it can be inferred from 'date of birth' but only if the dates are different
if len(set(sample_characteristics[5])) > 1:
    age_row = 5
else:
    age_row = None
    
# For 'gender', checking the unique values
if len(set(sample_characteristics[4])) > 1:
    gender_row = 4
else:
    gender_row = None

# 2.3 Data Type Conversion
def convert_trait(value):
    return None  # Not available

import datetime

def convert_age(value):
    try:
        birth_date = value.split(': ')[1]
        birth_date = datetime.datetime.strptime(birth_date, '%Y-%m-%d')
        age = datetime.datetime.now().year - birth_date.year
        return age
    except:
        return None

def convert_gender(value):
    try:
        gender = value.split(': ')[1].strip().lower()
        if gender == 'male':
            return 1
        elif gender == 'female':
            return 0
        else:
            return None
    except:
        return None

# 3. Save Metadata
save_cohort_info('GSE84046', './preprocessed/Language_Abilities/cohort_info.json', is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(
        clinical_data, 'Language_Abilities', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Language_Abilities/trait_data/GSE84046.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
